# Apresentação

O presente projeto foi originado no contexto das atividades da disciplina de pós-graduação [*Ciência e Visualização de Dados em Saúde*](https://ds4h.org), oferecida no primeiro semestre de 2022, na Unicamp.

> |Nome  | RA | Especialização|
> |--|--|--|
> | Edoarda  | 204057  | Computação |

# Contextualização da Proposta
> Apresentação da proposta de predição indicando os parâmetros adotados para a mesma com a justificativa (por que esses parâmetros foram adotados?).
> O ideal é que a proposta seja apresentada como uma pergunta de pesquisa.
> * Dado um paciente com alergia severa, quais são as chances da proxima vez que ele entrar no hospital seja morto? (ou esteja morto daqui a um mes algo do tipo)
  (meio dark mas esse com certeza ninguem fez!)
* Ou dado que uma pessoa tem alergias (pequena, media, severa), quais sao as chances da proxima vez que ela entrar no hospital vai ser por uma reacao alergica
* Parametro de predição: Iremos (tentar) predizer se uma pessoa tem mais chance de morrer em até 7 dias de uma visita ao ambulatório. 

  > **_NOTA:_**  A ideia inicial era usar algum desses: AVC, Encounter for symptom com descrição específica, reação alérgica ou quebra de osso. Porém, eles tinham muitos poucos dados (ou nenhum), e poderiam cair em uma versão menos extrema [deste](https://www.e-ir.info/2021/02/18/death-by-data-drones-kill-lists-and-algorithms/) problema.  
  
  * Dados utilizados: Alergias, Encontros*, Pacientes (datas de nascimento/morte)*

  > **_NOTA:_**  Em alergias, foi usado o campo severidade1, porque tinham dois campos de severidade com valores diferentes.
  > Checar se alergia afeta alguma coisa! entao veremos 

## Ferramentas
> Ferramentas Utilizadas:
 * Google Colab

> Pacotes Utilizados:
 * SciPy
 * NumPy
 * PANDAS
 * sklearn

# Metodologia
> Abordagem adotada pelo projeto na predição.
> Justificar as escolhas e (opcionalmente) apresentar fundamentos teóricos.

## Bases Adotadas para o Estudo

> Foram utilizadas as seguintes bases, oferecidas durante a própria disciplina:
* scenario01
* scenario02

As bases estão disponíveis [aqui!](https://github.com/santanche/lab2learn/tree/master/data/synthea)

# RASCUNHÃO (DELETAR ANTES DE ENTREGAR)
(dado k-1, sendo k a ultima visita, predizer k)

In [2]:
import sys
import scipy
import numpy
import matplotlib
import pandas as pd
import sklearn

from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
url = '/content/drive/MyDrive/synthea/scenario'

#scen01
patients01 = pd.read_csv(url + '01/csv/patients.csv')
allergies01 = pd.read_csv(url + '01/csv/allergies.csv')
allergies02 = pd.read_csv(url + '02/csv/allergies.csv')

encounters01 = pd.read_csv(url + '01/csv/encounters.csv')
encounters02 = pd.read_csv(url + '02/csv/encounters.csv')

allergies01['SEVERITY1'].unique()

array(['MODERATE', nan, 'MILD', 'SEVERE'], dtype=object)

  Enquanto estava explorando os dados do banco de alergias enquanto para popular a tabela que estava sendo preparada para a parte de Machine Learning, foi constado que alguns pacientes tinham Alergias em que não constava o grau de severidade. 

  Nesse caso, considerei que pacientes que tivessem uma alergia listada na tabela Alergias sem um valor especificado teriam ela em grau VERY MILD (consta alergia), e caso o paciente não estivesse por lá, considero que ele não tem essa alergia.

> **_NOTA:_**  Em questões de implementação, foi considerado:

  > | Tab.Alergias | Tab.P/ML |
  > |--|--|
  > | Não Consta | -1 |
  > | nan | 0 |
  > | 'MILD' | 1 |
  > | 'MODERATE' | 2 |
  > | 'SEVERE' | 3 |

In [5]:
def severidade(valor):
  if (valor == 'MILD'):
    return 1
  if (valor == 'MODERATE'):
    return 2
  if (valor == 'SEVERE'):
    return 3
  else:
    return 0

# Descobrindo as alergias únicas para popular a tabela
alergiahead = allergies01.drop_duplicates(subset=['CODE'], ignore_index=True)
# alergiahead #20 itens
alergiahead = pd.concat([alergiahead, allergies02.drop_duplicates(subset=['CODE'])], ignore_index=True)
alergiahead = alergiahead.drop_duplicates(subset=['CODE'], ignore_index=True)
alergiahead = alergiahead["DESCRIPTION"].values.tolist()
# alergiahead

# Adicionando nossas alergias na tabela pra poder mineirar!
encounters01 = pd.concat([encounters01, pd.DataFrame(columns=alergiahead)])
encounters02 = pd.concat([encounters02, pd.DataFrame(columns=alergiahead)])
# encounters01

# Populando os valores de acordo com as tabelas allergies!
# 0 -> Nan (Mas consta na tabela) 1 -> MILD 2 -> MODERATE 3 -> SEVERE

# pega pessoas alergicas para comparar
pessoas = pd.concat([allergies01.drop_duplicates(subset=['PATIENT']), allergies02.drop_duplicates(subset=['PATIENT'])], ignore_index=True)
pessoas = pessoas.drop_duplicates(subset=['PATIENT'], ignore_index=True)
pessoas = pessoas['PATIENT'].values.tolist()
#print(pessoas)
display(list(allergies01.columns.values))
# ['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'SYSTEM', 'DESCRIPTION',  'TYPE',  'CATEGORY',  'REACTION1',  'DESCRIPTION1',  'SEVERITY1',  'REACTION2',  'DESCRIPTION2', 'SEVERITY2']
# id  0,       1        2           3           4       5         6               7       8             9                 10          11                12          13              14

for alergico in pessoas:
  #print(alergico)
  # ----------------------------------------------------------------------------
  # TODO: separar isso para allergies01 e allergies02, ja que a gente ta fazendo um dataset so por vez
  # ----------------------------------------------------------------------------
  atchim = pd.concat([allergies01.loc[allergies01['PATIENT'] == alergico], allergies02.loc[allergies01['PATIENT'] == alergico]], ignore_index=True)
  #print(atchim)
  atchim = atchim.drop_duplicates()
  atchim = atchim.values.tolist()
  for linha in atchim:
    # paciente - id 2
    # description - id 6, associar a coluna (head)
    # severity1 - id 11, associar ao valor linha, col
    # associa linha em atchim e adiciona funcao em coluna especifca de alergia em tabela
    encounters01.loc[encounters01.PATIENT==linha[2], linha[6]] = severidade(linha[11])

['START',
 'STOP',
 'PATIENT',
 'ENCOUNTER',
 'CODE',
 'SYSTEM',
 'DESCRIPTION',
 'TYPE',
 'CATEGORY',
 'REACTION1',
 'DESCRIPTION1',
 'SEVERITY1',
 'REACTION2',
 'DESCRIPTION2',
 'SEVERITY2']

In [6]:
# arrumando a tabela pra so ter valores numericos pra gente poder mexer... sigh
# display(list(encounters01.columns.values))

encounters01 = encounters01.fillna(-1)
encounters01['START'] = pd.to_datetime(encounters01['START'], infer_datetime_format=True)
encounters01['STOP'] = pd.to_datetime(encounters01['STOP'], infer_datetime_format=True)

# ReasonDescription estava dando trabalho, se sobrar tempo arrasta de volta.
encounters01.drop('REASONDESCRIPTION', axis=1, inplace=True)

encounters01.to_csv('enc01clean.csv')

# ['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST',  'PAYER_COVERAGE',  'REASONCODE',  'REASONDESCRIPTION',
# transformar 0, 3-14 em escalares (o resto ja e numero) 15 se RD entrar
# -----------------------------------------------------------------------------
# NOTA DO FUTURO: É PRA TRANSOFORMAR TUDO EM ESCALAR 
# -----------------------------------------------------------------------------

# FUNÇÃO PERFEITA E MARAVILHOSA Q ME SALVOU
encounters01 = encounters01.convert_dtypes()
# print(encounters01.iloc[:, 3])

modelo = []
modelo.insert(0, preprocessing.LabelEncoder())
modelo[0].fit(encounters01.iloc[:, 0])
encounters01.iloc[:, 0] = modelo[0].transform(encounters01.iloc[:, 0])

# gambiarra por motivos que o insert do python n quer inserir direto no slot 3. Obg python. Perdi duas horas nisso.
modelo.append(None)
modelo.append(None)

for coluna in range(3, 14): 
  modelo.insert(coluna, preprocessing.LabelEncoder())
  modelo[coluna].fit(encounters01.iloc[:, coluna])
  encounters01.iloc[:, coluna] = modelo[coluna].transform(encounters01.iloc[:, coluna])

In [7]:
def fun(stamps):
  return (stamps - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")

encounters01['START'] = encounters01['START'].apply(fun)
encounters01['STOP'] = encounters01['STOP'].apply(fun)

# idade, #is dead

#todo mundo int! yay!
#display(encounters01.dtypes)

In [8]:
# https://medium.com/blog-do-zouza/seu-primeiro-projeto-de-machine-learning-em-python-passo-a-passo-78c5f7bce22d
# aparentemente int64 e Int64 sao tipos diferentes? Obg python. (2)
encounters01.info()

encounters01 = encounters01.astype('int64')
array = encounters01.values

X = array[:,0:4]
Y = array[:,4]
validation_size = 0.20
seed = 7
scoring = 'accuracy'
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Esse aqui foi para avaliar cada modelo pra testar, pode descomentar se quiser, mas ele não converge rápido (tudo menos SVM demorou uns 40min
# SVM rodou 20h e morreu)
"""
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn

results = []
names = []
for name, model in models:
   kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)
   cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
   results.append(cv_results)
   names.append(name)
   msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
   print(msg)
LR: 0.030178 (0.002590)
LDA: 0.073680 (0.002899)
KNN: 0.023642 (0.001959)
CART: 0.616026 (0.005204)
NB: 0.045120 (0.002648)
Tudo foi um lixo mas bora de CART (SVM só não terminou em 21h)
"""
# usar xgboost, ele costuma rodar legal em modelo que fica bom em CART - thales, parafraseado 

# Make predictions on validation dataset
mkart = DecisionTreeClassifier()
mkart.fit(X_train, Y_train)
predictions = mkart.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59481 entries, 0 to 59480
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype
---  ------                           --------------  -----
 0   Id                               59481 non-null  int64
 1   START                            59481 non-null  int64
 2   STOP                             59481 non-null  int64
 3   PATIENT                          59481 non-null  int64
 4   ORGANIZATION                     59481 non-null  int64
 5   PROVIDER                         59481 non-null  int64
 6   PAYER                            59481 non-null  int64
 7   ENCOUNTERCLASS                   59481 non-null  int64
 8   CODE                             59481 non-null  int64
 9   DESCRIPTION                      59481 non-null  int64
 10  BASE_ENCOUNTER_COST              59481 non-null  int64
 11  TOTAL_CLAIM_COST                 59481 non-null  int64
 12  PAYER_COVERAGE                   59481 non-nul

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [9]:
# Adicoes extras
# patients01['BIRTHDATE'] = patients01['BIRTHDATE'].apply((lambda x: age(x['BIRTHDATE'], pd.to_datetime("today")) if (x['DEATHDATE'] == numpy.nan) else age(x['BIRTHDATE'], x['DEATHDATE'])))

def age(born, death):
  born = pd.to_datetime(born)
  if (pd.isna(death)):
    today = pd.to_datetime("today")
  else: today = pd.to_datetime(death)
  return today.year - born.year - ((today.month, 
                                    today.day) < (born.month, 
                                                    born.day))


patients01 = patients01.convert_dtypes()
pacientes = pd.concat([patients01['Id'], patients01['BIRTHDATE']], axis=1)
pacientes = pd.concat([pacientes, patients01['DEATHDATE']], axis=1)
pacientesl = pacientes.values.tolist()

for linhas in range(0, len(pacientesl)):
  pacientesl[linhas][1] = age(pacientesl[linhas][1], pacientesl[linhas][2])
  if pd.isna(pacientesl[linhas][2]):
    pacientesl[linhas][2] = 1 # tá vivo
  else: pacientesl[linhas][2] = 0 # morreu

# Concatena lista nos encounters01, usando Id pra equalizar
# lembre q ta tudo normalizado!! entao vc tem q normalizar tb!!
# stop - start tb e converter em minutos...
'''
modelo = []
modelo.insert(0, preprocessing.LabelEncoder())
modelo[0].fit(encounters01.iloc[:, 0])
encounters01.iloc[:, 0] = modelo[0].transform(encounters01.iloc[:, 0])
'''
# modelo[0] tem os transformados do id de paciente, entao usar ele para id aqui...
# modelo [1] e [2] estao vazios, por idade e isDead neles (e criar um no final para o tempoconsulta)
# ia ficar muito mais facil só arrumar as coisas ali em cima, mas como esse é um trabalho que se quer saber
# como eu fui pensando ele vai ficar nu e cru assim mesmo. Dito isso, se esse codigo fosse para produção
# ele DEFINITIVAMENTE ia ser refatorado pq esse negócio é mais ineficiente que rapel horizontal

pacientesl = pd.DataFrame(pacientesl)

modelo.insert(1, preprocessing.LabelEncoder())
modelo[1].fit(pacientesl.iloc[:,1]) # idade
pacientesl.iloc[:, 1] = modelo[1].transform(pacientesl.iloc[:, 1])

modelo.insert(2, preprocessing.LabelEncoder())
modelo[2].fit(pacientesl.iloc[:,2]) # isDead
pacientesl.iloc[:, 2] = modelo[2].transform(pacientesl.iloc[:, 2])

#print(pacientesl.head()) # mudar Id para Pacient
#print(encounters01.head())
enc1bkp = pd.read_csv('enc01clean.csv')
pacientesl.to_csv("1_idadebkp.csv") 
encounters01['PATIENT'] = enc1bkp['PATIENT']
#encounters01 = modelo[3].inverse_transform(encounters01.iloc[:, 3])
pacientesl = pacientesl.rename(columns={0:"PATIENT"})
pacientesl = pacientesl.rename(columns={1:"AGE"})
pacientesl = pacientesl.rename(columns={2:"ISDEAD"})

#print(encounters01.head())
#print(pacientesl)
#print(modelo)
# n sei pq ele tá dando AttributeError: 'NoneType' object has no attribute 'fit'... garbage collection?
modelo.insert(3, preprocessing.LabelEncoder())
encounters01 = encounters01.merge(pacientesl, on='PATIENT', how='inner')
modelo[3].fit(encounters01.iloc[:, 3])
encounters01.iloc[:, 3] = modelo[3].transform(encounters01.iloc[:, 3])

def func(x):
    return int(x[2]) - int(x[1])

encounters01['START']  = encounters01.apply(func, axis = 1)
#print(encounters01)
encounters01.drop('STOP', axis=1, inplace=True)

modelo.append(preprocessing.LabelEncoder())
modelo[len(modelo)-1].fit(encounters01.iloc[:,1])
encounters01.iloc[:, 1] = modelo[len(modelo)-1].transform(encounters01.iloc[:, 1])

#quero birthdate, deathdate (as age, is dead)


In [19]:
print(encounters01)
print(pacientesl["AGE"].value_counts())
print(pacientesl["ISDEAD"].value_counts())
print("--------------------------------")
print(encounters01["AGE"].value_counts())
print(encounters01["ISDEAD"].value_counts())

          Id  START  PATIENT  ORGANIZATION  PROVIDER  PAYER  ENCOUNTERCLASS  \
0      13015      0      608           468       610      0               9   
1      28874      0      608           468       610      0               9   
2      14500      0      608           468       610      0               9   
3      54320      0      608           468       610      0               9   
4      58550      0      608           468       610      7               9   
...      ...    ...      ...           ...       ...    ...             ...   
59476  28393      0      660           381       180      7               0   
59477  46080      0      660           709       625      7               9   
59478  20277      0      660           381       180      7               0   
59479  52558      0      660            85       776      7               7   
59480  16955      0      660            85       776      7               7   

       CODE  DESCRIPTION  BASE_ENCOUNTER_COST  ... 

In [11]:
# tentativa 2
encounters01 = encounters01.astype('int64')
array = encounters01.values

X = array[:,0:4]
Y = array[:,4]
validation_size = 0.20
seed = 7
scoring = 'accuracy'
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

models = []
#models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
# models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn

results = []
names = []
for name, model in models:
   kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)
   cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
   results.append(cv_results)
   names.append(name)
   msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
   print(msg)

   '''
   4h run 11:28pm
   LR: 0.082801 (0.003065)
   LDA: 0.058822 (0.003260)
   KNN: 0.117897 (0.003506)
   CART: 0.994431 (0.001121)
   NB: 0.905262 (0.003190)
  '''

CART: 0.994326 (0.001275)
NB: 0.905262 (0.003190)


In [12]:
mkart = DecisionTreeClassifier()
mkart.fit(X_train, Y_train)
predictions = mkart.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
# Descomentar essa linha se quiser, mas é um print grande
# k q1cxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxdf]
#oooooiiiiiiiiiiiiiiiiprint(classification_report(Y_validation, predictions))

0.99504076657981
[[8 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 7 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         9
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         3
          11       1.00      1.00      1.00         5
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         4
          15       1.00      1.00      1.00         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

# Evolução do Projeto
> Nesse ponto (17/05) percebi que cometi alguns erros montando a matriz de encontros para treinamento.
> * Eu não tinha percebido que era necessario normalizar as datas, apenas convertê-las para um inteiro (UNIXtime). Por conta disso, os algoritmos de ML estão dando um peso ridiculamente grande para a data do Encontro, o que é errado.
> * Falando em datas, eu tinha dito que ia predizer a data da morte da pessoa mas esqueci de adicionar as datas de nascimento e morte dos pacientes (Oops.)
> * Além disso, os outros problemas de implementação provavelmente estão comentados nos comentários do código.

> * Com a extensão do prazo, normalizei todos o dados, adicionei a idade, e o estado de vida dos pacientes, e após rodei o avaliador de modelos de classificação novamente. 
> * A Árvore de Classificação teve *99.44%* de acurácia (!) e o Naive Bayes 90%.
 * Minha primeira impressão pessimista é que fiz algo errado outra vez.

# Discussão
> Fazer um breve debate sobre os resultados alcançados. Aqui pode ser feita a análise dos possíveis motivos que certos resultados foram alcançados. Por exemplo:
> * por que seu modelo alcançou (ou não) um bom resultado?
>> (17/05) O modelo provavelmente não alcançou um bom resultado pelos pontos dito acima, dito a não-normalização das datas e os pacientes imortais
>> Além disso, definitivamente faltou um bom manuseamento do tempo proposto.
> * por que o modelo de um cenário não se desempenhou bem em outro?
>
> Não é possivel tirar nenhuma conclusão de verdade com o modelo, visto que o melhor algoritmo só tinha 61% de acurácia. O estudo deve ser direcionado, dado uma extensão de prazo, à adicionar os campos que eu esqueci, normalizar o que faltou e testar outra vez para ver se temos resultados em que alguma coisa legal aconteça e aí ter uma direção mais legal (ou não).
> Depois de refazer o modelo, eu ainda não sei se pode ter sido tirada alguma conclusão, visto que ele teve uma acurácia boa demais para ser verdade e eu não sei onde exatamente eu errei.

> Como um amigo meu comentou, também seria possível treinar o modelo com XGBoost para ver se ele tem resultados melhores?

# Conclusão
> Destacar as principais conclusões obtidas no desenvolvimento do projeto.
>
> Destacar os principais desafios enfrentados.
>> Uma das principais dificuldades foi pensar em alguma questão de pesquisa que o Dataset pudesse tentar resolver. A outra grande dificuldade foi montar os dados de modo que um algoritmo de aprendizado de máquina de modo que ele rodasse (isso é em parte culpa minha, porque eu jurava que as implementações do python normalizavam ele na entrada. E eles acharem que Int64 e int64 são tipos diferentes (eu gastei mais tempo nisso do que eu gosto de admitir))
>
> Principais lições aprendidas:
  * NORMALIZE TODOS OS DADOS
  * Checagem de tipo de Python aparentemente é case-sensitive
>
> Trabalhos Futuros:
> * o que poderia ser melhorado se houvesse mais tempo?
>> Como dito anteriormente, treinar um modelo com um dataset com todas as informações necessárias (e normalizado) ia ser um bom começo.

# Referências Bibliográficas
> * [Seu primeiro Projeto de Machine Learning em Python (Passo a Passo)](https://medium.com/blog-do-zouza/seu-primeiro-projeto-de-machine-learning-em-python-passo-a-passo-78c5f7bce22d)
> * [How to Make Predictions with scikit-learn](https://machinelearningmastery.com/make-predictions-scikit-learn/)
> * [Scikit Learn User Guide](https://scikit-learn.org/stable/user_guide.html#user-guide)
> * [SnomedCT Code Finder](https://www.findacode.com/search/search.php)
> * [Teste](https://www.geeksforgeeks.org/disease-prediction-using-machine-learning/)

# Agradecimentos Extras
> * [StackOverflow](https://stackoverflow.com/)
> * Thales, que me tirou muitas dúvidas de ML